In [1]:
def text2paragraphs(filename, min_size=1):
    """ A text contained in the file 'filename' will be read 
    and chopped into paragraphs.
    Paragraphs with a string length less than min_size will be ignored.
    A list of paragraph strings will be returned"""
    
    txt = open(filename).read()
    paragraphs = [para for para in txt.split("\n\n") if len(para) > min_size]
    return paragraphs


In [2]:
# the position of lables is very important
# it corresponds to a novel by that author within "files"
# the position of the author is also relevant, as it will correspond to metrics
# i.e. Samuel Butler's metrics are always returned in position 1
labels = ['Virginia Woolf', 'Samuel Butler', 'Herman Melville', 
          'David Herbert Lawrence', 'Daniel Defoe', 'James Joyce']


# names of books we have to train our machine model
files = ['night_and_day_virginia_woolf.txt', 'the_way_of_all_flash_butler.txt',
         'moby_dick_melville.txt', 'sons_and_lovers_lawrence.txt',
         'robinson_crusoe_defoe.txt', 'james_joyce_ulysses.txt']

# location of our books
path = "/home/student/mycode/natural/books/"


In [3]:
data = []
targets = []
counter = 0

# loop across all files we have downloaded
for fname in files:
    paras = text2paragraphs(path + fname, min_size=150) # return a book with paragraphs over 150 chars in a list
    data.extend(paras)
    targets += [counter] * len(paras)
    counter += 1


In [9]:
# cell is useless, because train_test_split will do the shuffling!

import random

data_targets = list(zip(data, targets))
# splay (data_targets)
# create random permutation on list:
data_targets = random.sample(data_targets, len(data_targets))

data, targets = list(zip(*data_targets))


In [10]:
from sklearn.model_selection import train_test_split

res = train_test_split(data, targets, 
                       train_size=0.8,
                       test_size=0.2,
                       random_state=42)
train_data, test_data, train_targets, test_targets = res 


In [11]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

vectorizer = CountVectorizer()

vectors = vectorizer.fit_transform(train_data)

# creating a classifier
classifier = MultinomialNB(alpha=.01)
classifier.fit(vectors, train_targets)

vectors_test = vectorizer.transform(test_data)

predictions = classifier.predict(vectors_test)
accuracy_score = metrics.accuracy_score(test_targets, 
                                        predictions)
f1_score = metrics.f1_score(test_targets, 
                            predictions, 
                            average='macro')

print("accuracy score: ", accuracy_score)
print("F1-score: ", f1_score)


accuracy score:  0.9183451279259662
F1-score:  0.9138997235426528


In [12]:
# we want to use paragraphs from this 2nd Virginia Wolf 
paras = text2paragraphs(path + "the_voyage_out_virginia_woolf.txt", min_size=250)

# start on paragraph 100 and go to paragraph 500
first_para, last_para = 100, 500
vectors_test = vectorizer.transform(paras[first_para: last_para]) # pass a list of strings that will be used to make predictions against
#vectors_test = vectorizer.transform(["To be or not to be"])

predictions = classifier.predict(vectors_test) # make our predictions
print(predictions)
targets = [0] * (last_para - first_para)
accuracy_score = metrics.accuracy_score(targets, 
                                        predictions)
precision_score = metrics.precision_score(targets, 
                                          predictions, 
                                          average='macro')

f1_score = metrics.f1_score(targets, 
                            predictions, 
                            average='macro')

print("accuracy score: ", accuracy_score)
print("precision score: ", accuracy_score)
print("F1-score: ", f1_score)


[5 0 0 5 0 0 5 0 2 5 0 0 0 0 0 0 0 0 1 1 1 0 0 5 1 5 0 0 1 0 0 0 5 0 2 3 0
 2 2 3 0 0 3 5 0 3 0 0 0 0 0 0 2 5 2 0 0 0 0 1 0 0 5 0 0 0 0 0 0 0 5 2 0 0
 0 0 1 0 0 2 2 5 0 2 2 0 5 0 0 5 0 0 0 0 0 5 0 0 1 0 0 3 5 5 5 5 5 5 5 0 1
 0 0 0 0 0 0 1 3 0 0 0 2 0 1 2 2 2 5 5 0 2 0 5 5 0 0 5 1 1 1 0 0 0 5 0 0 0
 0 0 5 1 0 0 5 1 1 0 1 0 0 0 0 0 0 5 0 1 5 0 0 0 5 5 5 0 2 0 0 0 0 0 0 0 0
 2 0 0 0 0 0 1 0 0 0 3 0 0 0 0 0 0 0 5 0 0 0 0 5 5 5 2 0 5 0 2 3 0 0 0 5 0
 0 5 2 0 0 0 0 0 3 0 0 0 0 2 0 0 5 3 5 1 0 1 5 5 0 0 5 0 1 1 1 0 0 0 0 1 3
 1 1 0 5 0 5 5 2 0 0 0 0 5 0 2 2 0 5 0 0 0 0 0 0 3 0 4 0 0 0 0 1 0 0 0 0 1
 0 0 2 5 5 0 5 0 0 3 3 0 5 3 0 0 0 0 0 1 3 0 0 0 0 1 0 0 0 0 3 0 0 0 0 0 5
 0 3 5 0 5 5 0 0 1 1 1 0 0 0 0 2 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0
 1 1 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0]
accuracy score:  0.6275
precision score:  0.6275
F1-score:  0.12852022529441884


In [13]:
# perform a probability test
predictions = classifier.predict_proba(vectors_test)
print(predictions)


[[1.28958409e-006 5.41514900e-008 2.74244416e-008 1.12368554e-010
  2.63112149e-014 9.99998629e-001]
 [1.00000000e+000 6.12169406e-011 6.52293689e-013 2.07262711e-016
  1.37017266e-025 9.09129406e-012]
 [9.99929690e-001 7.66152856e-014 2.52784354e-016 5.26982841e-012
  4.41437976e-028 7.03096848e-005]
 ...
 [9.99999996e-001 3.55281162e-009 4.46287208e-054 1.94913095e-027
  1.24433910e-051 3.85868741e-043]
 [9.90287105e-001 1.14141860e-019 7.11458544e-023 6.14294774e-030
  1.44917768e-061 9.71289548e-003]
 [1.00000000e+000 6.56852768e-066 6.20125050e-087 2.92990215e-102
  1.37836618e-134 1.70523802e-070]]


In [14]:
for i in range(0, 10):
    print(predictions[i], paras[i+first_para])


[1.28958409e-06 5.41514900e-08 2.74244416e-08 1.12368554e-10
 2.63112149e-14 9.99998629e-01] "That's the painful thing about pets," said Mr. Dalloway; "they die. The
first sorrow I can remember was for the death of a dormouse. I regret to
say that I sat upon it. Still, that didn't make one any the less sorry.
Here lies the duck that Samuel Johnson sat on, eh? I was big for my
age."
[1.00000000e+00 6.12169406e-11 6.52293689e-13 2.07262711e-16
 1.37017266e-25 9.09129406e-12] "Please tell me--everything." That was what she wanted to say. He had
drawn apart one little chink and showed astonishing treasures. It seemed
to her incredible that a man like that should be willing to talk to her.
He had sisters and pets, and once lived in the country. She stirred her
tea round and round; the bubbles which swam and clustered in the cup
seemed to her like the union of their minds.
[9.99929690e-01 7.66152856e-14 2.52784354e-16 5.26982841e-12
 4.41437976e-28 7.03096848e-05] The talk meanwhile raced pa